### Name: Mukund Tamizharasan
### USC ID: 7355725345

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
%cd '/content/drive/MyDrive/DSCI552_HW7/'

/content/drive/MyDrive/DSCI552_HW7


# Generative Models for Text

In [49]:
import os
import tensorflow as tf
import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, Callback

In [50]:
import re


### 1. (c) i. Concatenate your text files to create a corpus of Russell’s writings.

In [51]:
corpus = []
for book in os.listdir("./books/"):
    f = open("./books/"+book, 'r', errors='ignore',encoding = "ascii")
    text = f.read()
    text = text.replace("\n","")
    text = text.lower()
    text = re.sub('[^A-Za-z0-9 ,.]+', '', text)
    corpus.append(text)

corpus = sorted(corpus, key=lambda x : len(x))

### 1. (c) ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters.

In [52]:
full_text = ''
for text in corpus[:4]:
    full_text+=text
chars = list(set(full_text))

chars2ascii = {char:ord(char) for char in chars}
ascii2chars = {ord(char):char for char in chars}

In [53]:
print(chars2ascii)

{'n': 110, '5': 53, '6': 54, 'h': 104, '4': 52, 'w': 119, 'a': 97, 'l': 108, 'j': 106, 'u': 117, 'y': 121, 'f': 102, 'v': 118, '8': 56, '7': 55, 'x': 120, 'z': 122, ',': 44, '3': 51, '9': 57, 'm': 109, 'b': 98, 'g': 103, 'o': 111, '.': 46, 't': 116, ' ': 32, 'c': 99, 'q': 113, '1': 49, 'e': 101, 'p': 112, 'i': 105, 'd': 100, '0': 48, 's': 115, '2': 50, 'r': 114, 'k': 107}


### 1. (c) iii. Choose a window size, e.g., W = 100.

In [54]:
W = 100

print("Length of corpus:", len(full_text))
n = max(chars2ascii.values())

Length of corpus: 1536315


### 1. (c) iv. Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence.

In [55]:
x = []
y = []
for i in range(len(full_text)-W+1):
    temp = []
    for j in range(W-1):
        temp.append(chars2ascii[full_text[i+j]]/n)
    x.append(temp)
    y.append(chars2ascii[full_text[i+j+1]])
x = np.array(x)

### 1. (c) v. Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements.

In [56]:
y_oh = to_categorical(y)
y_oh.shape

(1536216, 123)

In [57]:
x.shape

(1536216, 99)

In [58]:
x=x.reshape(1536216, 99, 1)
x.shape

(1536216, 99, 1)

### 1. (c) vi. Use a single hidden layer for the LSTM with N = 256 (or less) memory units.

In [59]:
# References:
# https://www.analyticsvidhya.com/blog/2021/06/lstm-for-text-classification/

model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1],x.shape[2]), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(123,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 123)               31611     
                                                                 
Total params: 295,803
Trainable params: 295,803
Non-trainable params: 0
_________________________________________________________________


### 1. (c) x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

In [ ]:
callback = [ModelCheckpoint(filepath='./best_weights_lstm.h5',monitor="loss",verbose=1,save_best_only=True, save_freq="epoch"), ]
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x, y_oh, batch_size=512, epochs=30, callbacks=[callback])

Epoch 1/30
758/758 [==============================] - ETA: 0s - loss: 57467301888.0000 - accuracy: 0.0857
Epoch 00001: loss improved from inf to 57467301888.00000, saving model to ./best_weights_lstm.h5
758/758 [==============================] - 337s 442ms/step - loss: 57467301888.0000 - accuracy: 0.0857
Epoch 2/30
758/758 [==============================] - ETA: 0s - loss: 845721.9375 - accuracy: 0.1514
Epoch 00002: loss improved from 57467301888.00000 to 845721.93750, saving model to ./best_weights_lstm.h5
758/758 [==============================] - 330s 435ms/step - loss: 845721.9375 - accuracy: 0.1514
Epoch 3/30
758/758 [==============================] - ETA: 0s - loss: 2.9611 - accuracy: 0.1603
Epoch 00003: loss improved from 845721.93750 to 2.96106, saving model to ./best_weights_lstm.h5
758/758 [==============================] - 330s 435ms/step - loss: 2.9611 - accuracy: 0.1603
Epoch 4/30
758/758 [==============================] - ETA: 0s - loss: 2.9592 - accuracy: 0.1603
Epoch 00

In [60]:
best_model = keras.models.load_model("./best_weights_lstm.h5")

### 1. (c) xi. Use the network with the best weights to generate 1000 characters, using the following text as initialization of the network:

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.

In [72]:
test_text = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object."

test_textenc = [chars2ascii[i] for i in test_text[-99:].lower()]
test_textenc = np.array(test_textenc)
test_textenc = test_textenc.reshape(1,99,1)

char_keys=list(ascii2chars.keys())

for i in range(1000):
  pred = list(model.predict(test_textenc)[0])
  pred_ = [[pred[i],i] for i in range(len(pred))]
  pred_ = sorted(pred_,key=lambda x: x[0],reverse=True)
  char =''
  for i in pred_:
    if i[1] in char_keys:
      char = i[1]
      break
  test_text+=ascii2chars[char]
  temp = list(test_textenc.reshape(1,99)[0])
  temp.append(char)
  temp = np.array(temp[1:])
  test_textenc = temp.reshape(1,99,1)


In [73]:
test_text

'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.6 v7   gm     z5     m pche c am  w  q a      2 z  5m     ek   3 m 4  53e  e   0   o  z l  .   4 g3q gl7  ze  5 heg jq     6m  56  4qhg      z6z  6ee y  y  kzqq z  m  h 6 gn bhehm64 e k  6e      v. mq   j 3 ohm 2  bqc 6cztq 5  coqe5y56v 1h c 5gmbkz  g6   zm 3 9   6je5 e tk v  pb z5zl z    4  e  zzj7ec 6 ot  meqz9o3 m   z.6z j   g   qd m e2  6u m    46 e et   3 z93  8  e0p2z vqg     zgt g1lm    ma6e c  cw    17e 6mh  e  z  q  w j  m z s  54bme   . u w 7c  g z3 0t6o6y  p 7 m  6 o t  wq3 6  66   pc z 5 q23  q  j  t   g   8  5w   m7  z          mq   e  3r  e g pht6 z   em6m zl. v  5 3   e1 5ew  m      z     6 e 7m   z  6 47z 4z o z    j 5 z  g u   v g vt z c zez3b    4azp e  z         b46 52mc e v   m omtbz  g  6a  q5m   65g cmtk zm  c     k 3 e6zz 5 czt   z.   mz   t  6m t  e 6m   ta h5z4cotgh2vg96 p   mzj       2gv  jmh9 5g 6 l7 2 g  cz